# 03: Multi-Turn Conversations & Evaluation

This notebook demonstrates multi-turn conversation management and
how to evaluate the agent on the DeepSearchQA benchmark.

## Learning Objectives

- Manage multi-turn conversations with session state
- Use the `DeepSearchQAEvaluator` for systematic evaluation
- Analyze evaluation results with rich visualizations
- Understand evaluation metrics for research agents

In [ ]:
# Setup: Load environment and configure rich console
from aieng.agent_evals import (
    create_console,
    display_evaluation_result,
    display_info,
    display_metrics_table,
    display_success,
)
from dotenv import load_dotenv


console = create_console()
load_dotenv(verbose=True)

## 1. Multi-Turn Session Management

In [ ]:
from aieng.agent_evals.knowledge_agent import ConversationSession
from rich.panel import Panel
from rich.table import Table


# Create a session
session = ConversationSession()

console.print(
    Panel(
        f"[cyan]Session ID:[/cyan] {session.session_id}",
        title="🗨️ New Session Created",
        border_style="green",
    )
)

In [ ]:
# Simulate a multi-turn conversation
session.add_user_message("What is the capital of France?")
session.add_assistant_message("The capital of France is Paris.")

session.add_user_message("What is its population?")
session.add_assistant_message("Paris has a population of about 2.1 million in the city proper.")

# Display the conversation
console.print("[bold]📝 Conversation History[/bold]\n")
console.print(Panel(session.get_history_as_text(), border_style="blue"))

In [ ]:
# Get history as structured data
history = session.get_history()

history_table = Table(title="📊 Structured History", show_header=True, header_style="bold cyan")
history_table.add_column("Role", style="cyan")
history_table.add_column("Content", style="white")

for msg in history:
    role_style = "green" if msg["role"] == "user" else "blue"
    history_table.add_row(
        f"[{role_style}]{msg['role'].upper()}[/{role_style}]",
        msg["content"][:50] + "..." if len(msg["content"]) > 50 else msg["content"],
    )

console.print(history_table)

## 2. Using Sessions with Gradio State

In Gradio apps, use `get_or_create_session` to manage sessions.

In [ ]:
from aieng.agent_evals.knowledge_agent import get_or_create_session


# Simulate Gradio state
gradio_state = {}

# First turn - creates a new session
session1 = get_or_create_session(gradio_state)
session1.add_user_message("Hello!")
display_info(f"Created session: {session1.session_id}", console=console)

# Second turn - retrieves existing session
session2 = get_or_create_session(gradio_state)
display_info(f"Retrieved session: {session2.session_id}", console=console)
display_success(f"Same session: {session1 is session2}", console=console)
display_info(f"Messages in session: {len(session2)}", console=console)

## 3. Running DeepSearchQA Evaluation

The `DeepSearchQAEvaluator` provides a systematic way to evaluate the agent.

In [ ]:
from aieng.agent_evals.knowledge_agent import (
    DeepSearchQADataset,
    DeepSearchQAEvaluator,
    KnowledgeGroundedAgent,
)


# Create agent and evaluator
with console.status("[cyan]Initializing agent and evaluator...[/cyan]", spinner="dots"):
    agent = KnowledgeGroundedAgent()
    evaluator = DeepSearchQAEvaluator(agent)

display_success(f"Dataset size: {len(evaluator.dataset)} examples", console=console)

In [ ]:
# Evaluate a small sample
console.print("[bold]🔬 Running evaluation on 3 examples...[/bold]\n")

console.print("[dim]Evaluating...[/dim]")
results = await evaluator.evaluate_sample_async(n=3, random_state=42)

display_success(f"Completed {len(results)} evaluations", console=console)

In [ ]:
# View results using the display utility
console.print("\n[bold]📋 Evaluation Results[/bold]\n")

for result in results:
    contains_answer = result.ground_truth.lower() in result.prediction.lower()
    display_evaluation_result(
        example_id=result.example_id,
        problem=result.problem,
        ground_truth=result.ground_truth,
        prediction=result.prediction,
        sources_used=result.sources_used,
        search_queries=result.search_queries,
        contains_answer=contains_answer,
        console=console,
    )

## 4. Analyzing Evaluation Results

In [ ]:
# Convert to DataFrame for analysis
df = evaluator.results_to_dataframe(results)

# Calculate metrics
containment_correct = sum(1 for r in results if r.ground_truth.lower() in r.prediction.lower())
containment_accuracy = containment_correct / len(results) * 100

metrics = {
    "Total Examples": len(results),
    "Containment Accuracy": f"{containment_accuracy:.1f}%",
    "Avg Sources Used": df["sources_used"].mean(),
    "Avg Search Queries": df["search_queries"].apply(len).mean(),
}

display_metrics_table(metrics, title="Evaluation Metrics", console=console)

## 5. Understanding Evaluation Metrics

For research agents, we care about:

1. **Answer Correctness**: Does the prediction match the ground truth?
2. **Source Quality**: Are the sources relevant and authoritative?
3. **Comprehensiveness**: Did the agent find all necessary information?
4. **Search Efficiency**: How many searches were needed?

DeepSearchQA specifically measures:
- **Precision**: Quality of the answer
- **Recall**: Completeness of the answer (for list-type questions)

In [ ]:
# Manual correctness check with better display
def check_answer_contains_ground_truth(prediction: str, ground_truth: str) -> bool:
    """Check if prediction contains the ground truth answer."""
    return ground_truth.lower() in prediction.lower()


# Check our results
console.print("\n[bold]📊 Correctness Check[/bold]\n")

result_table = Table(show_header=True, header_style="bold cyan")
result_table.add_column("Example", style="cyan")
result_table.add_column("Status", style="white")
result_table.add_column("Expected", style="dim")

for result in results:
    contains = check_answer_contains_ground_truth(result.prediction, result.ground_truth)
    status = "[green]✓ MATCH[/green]" if contains else "[yellow]✗ NO MATCH[/yellow]"
    result_table.add_row(
        str(result.example_id),
        status,
        result.ground_truth[:40] + "..." if len(result.ground_truth) > 40 else result.ground_truth,
    )

console.print(result_table)

## 6. Exploring Categories

In [ ]:
# Get examples from a specific category
dataset = DeepSearchQADataset()
categories = dataset.get_categories()

cat_table = Table(title="📁 Available Categories", show_header=True, header_style="bold green")
cat_table.add_column("Category", style="white")
cat_table.add_column("Count", style="cyan", justify="right")

for cat in sorted(categories):
    count = len(dataset.get_by_category(cat))
    cat_table.add_row(cat, str(count))

console.print(cat_table)

## Summary

In this notebook, you learned:

1. How to manage multi-turn conversations with `ConversationSession`
2. How to use `get_or_create_session` for Gradio integration
3. How to run systematic evaluations with `DeepSearchQAEvaluator`
4. How to analyze evaluation results with rich visualizations
5. Key metrics for evaluating research agents

## Next Steps

- Run the Gradio app for interactive testing
- Experiment with different models (gemini-2.5-pro vs flash)
- Try the async evaluator for larger-scale evaluation
- Implement LLM-as-judge evaluation for more nuanced correctness checking

In [ ]:
console.print(
    Panel(
        "[green]✓[/green] Notebook complete!\n\n"
        "[cyan]Next:[/cyan] Run [bold]gradio_app.py[/bold] for interactive testing.",
        title="🎉 Done",
        border_style="green",
    )
)